In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi as pi
from numpy import exp as exp
from numpy import cos as cos
from numpy import sin as sin
from tqdm import tqdm
from skimage.feature import hog
from scipy.ndimage import sobel

# 弹性纵横网络

In [2]:
def cutline(image,sub1=None,sub2=None,sub3=None,sub4=None,mode='just cut',t=6):         #注意！image必须要是0-1反转过的0-1二值图片！
    #statistics=[]
    #image = pic.copy()
    #img = binaryImage(image)
    #plant = plant != 255
    #plant = image.astype(int)
    allPoint = np.sum(image)
    if allPoint==0:
        return np.zeros(t**2)
    perline = allPoint / t
    lie = np.sum(image,axis = 0)
    hang = np.sum(image,axis = 1)
    
    for i in range(1,len(lie)):
        lie[i] = lie[i-1] + lie[i]
    for i in range(1,len(hang)):
        hang[i] = hang[i-1] + hang[i]
        
    lieline = [0] * t + [image.shape[1]]
    hangline = [0] * t + [image.shape[0]]
   
    for i in range(0,t-1):
        lieline[i+1] = np.argmin(abs(lie - perline * (i + 1))) + 1
        #print(np.argmin(abs(lie - perline * (i + 1))) + 1)
        hangline[i+1] = np.argmin(abs(hang - perline * (i + 1))) + 1
    
    if mode=='just cut':
        return lieline,hangline
        
    if mode=='statistics':
        statistics=np.zeros(t**2)

        for j in range(t):
            for k in range(t):
                region=image[hangline[j]:hangline[j+1],lieline[k]:lieline[k+1]]
                statistics[j*t+k]=np.sum(region)/allPoint
        return statistics
    
    if mode=='sub-statistics':
        statistics=np.zeros(t**2*4)
        for j in range(t):
            for k in range(t):
                region1=sub1[hangline[j]:hangline[j+1],lieline[k]:lieline[k+1]]
                region2=sub2[hangline[j]:hangline[j+1],lieline[k]:lieline[k+1]]
                region3=sub3[hangline[j]:hangline[j+1],lieline[k]:lieline[k+1]]
                region4=sub4[hangline[j]:hangline[j+1],lieline[k]:lieline[k+1]]
                
                statistics[j*t+k]=np.sum(region1)/allPoint
                statistics[t**2+j*t+k]=np.sum(region2)/allPoint
                statistics[2*t**2+j*t+k]=np.sum(region2)/allPoint
                statistics[3*t**2+j*t+k]=np.sum(region2)/allPoint
                
        return statistics

# 子笔划提取-特征提取1

In [3]:
def makeCountLength(img,mode01 = False):
    img = img.astype(int)
    arr = np.pad(img, pad_width=1, mode='constant', constant_values=0)
    result = np.zeros((img.shape[0],img.shape[1],4))
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j] == 0:
                continue
                
            #设置横
            if j == 0 or img[i,j-1] == 0:
                result[i,j,0] = np.argmin(arr[i+1,j+1:])
            else:
                result[i,j,0] = result[i,j-1,0]
            
            #设置竖
            if i == 0 or img[i-1,j] == 0:
                result[i,j,2] = np.argmin(arr[i+1:,j+1])
                
            else:
                result[i,j,2] = result[i-1,j,2]
                
                
            #设置撇
            if i == 0 or j == img.shape[1] - 1 or img[i-1,j+1] == 0:
                result[i,j,1] = np.argmin(np.fliplr(arr[i+1:,:j+2]).diagonal())
            else:
                result[i,j,1] = result[i-1,j+1,1]
                
            #设置捺
            if i == 0 or j == 0 or img[i-1,j-1] == 0:
                result[i,j,3] = np.argmin(arr[i+1:,j+1:].diagonal())
            else:
                result[i,j,3] = result[i-1,j-1,3]
    
     
    #加上上下两个
    temp = result.copy()
    result[1:,1:] += temp[:img.shape[0] - 1,:img.shape[1] - 1]
    result[:img.shape[0] - 1,:img.shape[1] - 1] += temp[1:,1:]
    result[1:,:img.shape[1] - 1] += temp[:img.shape[0] - 1,1:]
    result[:img.shape[0] - 1,1:] += temp[1:,:img.shape[0] - 1]
    
    #return result
    
    #取最大值
    temp = np.max(result,axis = 2)
    r = np.zeros_like(result)
    for i in range(4):
        r[:,:,i] = np.equal(result[:,:,i], temp)

    temp = np.equal(temp,0).astype(int)
    for i in range(4):
        r[:,:,i] -= temp

    
    #return r
        
        
    #取前后
    result = r.copy()
    result[1:,:,0] += r[:img.shape[0]-1,:,0]
    result[:img.shape[0]-1,:,0] += r[1:,:,0]
    
    result[1:,1:,1] += r[:img.shape[0] - 1,:img.shape[1] - 1,1]
    result[:img.shape[0] - 1,:img.shape[1] - 1,1] += r[1:,1:,1]
    
    result[:,1:,2] += r[:,:img.shape[0]-1,2]
    result[:,:img.shape[0]-1,2] += r[:,1:,2]
    
    result[1:,:img.shape[1] - 1,3] += r[:img.shape[0] - 1,1:,3]
    result[:img.shape[0] - 1,1:,3] += r[1:,:img.shape[0] - 1,3]
    
    
    #先加起来，只要大于零就代表有，因此取bool值再反转即可
    result = result.astype(bool) == False
    
    if mode01:
        return (result==0).astype(int)
    
    result = result.astype(np.uint8) * 255
                
    return result

In [4]:
def statistics_strokes_extraction(img,t=6):
    result = makeCountLength(img, mode01 = True)
    heng,shu,pie,na = result[:,:,0],result[:,:,2],result[:,:,1],result[:,:,3]
    strokes_stats = cutline(img,heng,shu,pie,na,'sub-statistics')
    return strokes_stats

# 内外轮廓+hog-特征提取2

In [5]:
def extract_feature(img):
    h, w = img.shape

    # 内外轮廓特征
    feature1 = np.zeros(32)
    # 上下
    for cnt in range(4):
        p1 = p2 = p3 = p4 = 0
        for j in range(int(w / 4 * cnt), int(w / 4 * (cnt + 1))):
            # 上
            i = 0
            while img[i, j] == 0 and i < h - 1:
                p1 += 1
                i += 1
            while img[i, j] == 1 and i < h - 1:
                i += 1
            while img[i, j] == 0 and i < h - 1:
                p2 += 1
                i += 1
            # 下
            i = h - 1
            while img[i, j] == 0 and i > 0:
                p3 += 1
                i -= 1
            while img[i, j] == 1 and i > 0:
                i -= 1
            while img[i, j] == 0 and i > 0:
                p4 += 1
                i -= 1
        feature1[cnt] = p1
        feature1[cnt + 16] = p2
        feature1[cnt + 4] = p3
        feature1[cnt + 20] = p4

    # 左右
    for cnt in range(8, 12):
        p1 = p2 = p3 = p4 = 0
        for i in range(int(h / 4 * (cnt - 8)), int(h / 4 * (cnt - 7))):
            # 左
            j = 0
            while img[i, j] == 0 and j < w - 1:
                p1 += 1
                j += 1
            while img[i, j] == 1 and j < w - 1:
                j += 1
            while img[i, j] == 0 and j < w - 1:
                p2 += 1
                j += 1
            # 右
            j = w - 1
            while img[i, j] == 0 and j > 0:
                p3 += 1
                j -= 1
            while img[i, j] == 1 and j > 0:
                j -= 1
            while img[i, j] == 0 and j > 0:
                p4 += 1
                j -= 1
        feature1[cnt] = p1
        feature1[cnt + 16] = p2
        feature1[cnt + 4] = p3
        feature1[cnt + 20] = p4

    # HoG特征
    feature2 = hog(img, pixels_per_cell=(160,160), cells_per_block=(2, 2), orientations=9)
    feature2 = feature2.flatten()
    combined_feature = np.concatenate([feature1, feature2])

    return combined_feature

# gabor-特征提取3

In [6]:
def build_fixed_size_gabor_filters(ksize, thetas, lambd, sigma, gamma, psi):
    filters = []
    for theta in thetas:
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filters.append(kern)
    return filters

def gabor_extraction(image, filters, t=5):
    hang, lie = cutline(image, t=t)
    feature = np.zeros(t * t * len(filters))

    for i in range(t):
        for j in range(t):
            region = image[hang[i]:hang[i+1], lie[j]:lie[j+1]]
            if region.shape[0] == 0 or region.shape[1] == 0:
                continue
            region = region.astype('float32')
            for k, kern in enumerate(filters):
                fimg = cv2.filter2D(region, cv2.CV_8UC3, kern)
                feature_idx = i * t * len(filters) + j * len(filters) + k
                feature[feature_idx] = np.sum(fimg)

    return feature

# 创建固定大小的Gabor滤波器
ksize = 21 # 你可以根据需要调整这个大小
thetas = np.arange(0, np.pi, np.pi / 4)
lambd = 10.0
sigma = 5.0
gamma = 0.5
psi = 0
filters = build_fixed_size_gabor_filters(ksize, thetas, lambd, sigma, gamma, psi)

# gabor双弹性网络-特征提取4

In [7]:
def build_fixed_size_gabor_filters(ksize, thetas, lambd, sigma, gamma, psi):
    filters = []
    for theta in thetas:
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filters.append(kern)
    return filters

def gabor_extraction(image, filters, t=5):
    hang, lie = cutline(image, t=t, mode='just cut')
    feature = np.zeros(t * t * len(filters))

    for i in range(t):
        for j in range(t):
            region = image[hang[i]:hang[i+1], lie[j]:lie[j+1]]
            if region.shape[0] == 0 or region.shape[1] == 0:
                continue
            region = region.astype('float32')
            for k, kern in enumerate(filters):
                fimg = cv2.filter2D(region, cv2.CV_8UC3, kern)
                feature_idx = i * t * len(filters) + j * len(filters) + k
                feature[feature_idx] = np.sum(fimg)

    return feature

# 创建固定大小的Gabor滤波器
ksize = 21 # 你可以根据需要调整这个大小
thetas = np.arange(0, np.pi, np.pi / 4)
lambd = 10.0
sigma = 5.0
gamma = 0.5
psi = 0
filters = build_fixed_size_gabor_filters(ksize, thetas, lambd, sigma, gamma, psi)

def turn45(img):
    # 获取图像中心坐标
    center = (img.shape[1] // 2, img.shape[0] // 2)
    
    # 设置旋转矩阵
    rotation_matrix = cv2.getRotationMatrix2D(center, 45, 1.0)
    
    # 旋转图像
    rotated_image = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]), borderValue=(255,))

    return rotated_image


def gabor_ZiBiHua(img):
    image = (img == False).astype(np.uint8) * 255

    ZBH = makeCountLength(image)
    feature = np.array([])
    for i in range(4):
        if i % 2 == 0:
            pic = (ZBH[:,:,i] == 0).astype(int)
            feature = np.append(feature,gabor_extraction(pic,filters,t = 5))
        else:
            pic = (turn45(ZBH[:,:,i]) == 0).astype(int)
            feature = np.append(feature,gabor_extraction(pic,filters,t = 3))
    return feature

# hough特征-特征提取5

In [8]:
thetas = np.linspace(-np.pi/2, np.pi/2, 128)
coses= np.cos(thetas)
sins=np.sin(thetas)
theta_intervals=np.array([0, 18,36,54,73,91,109,128])      #将hough累计器从(140,128)降维到(16,7)时，对theta轴的划分

def hough_extraction(image,N=8):
    # 计算重心
    rows, cols = np.where(image == 1)  # 找到所有黑色像素的位置
    centroid = (round(np.mean(rows)),round(np.mean(cols)))
    # 计算所有黑色像素点到重心的距离
    distances = np.sqrt((cols - centroid[1])**2 + (rows - centroid[0])**2)
    # 找到最大距离
    max_distance = np.max(distances)
    max_radius = int(np.floor(max_distance)+1)
    black_pixel_counts = np.zeros(max_radius)
    for radius in range(1, max_radius + 1):
        # 计算当前半径内的黑色像素数量
        count = np.sum(np.sqrt((cols - centroid[0])**2 + (rows - centroid[1])**2) <= radius )
        black_pixel_counts[radius-1]=count
    total=np.sum(image)
    perline = total / N  
    ideal_radii=np.zeros(2*N+1)
    ideal_radii[0]=-max_radius
    ideal_radii[2*N]=max_radius
    # 循环以找到N-1个合适的弹性分割半径
    for i in range(N-1):
        # 找到最接近目标差异数累积的半径
        ideal_radii[N+i+1]= np.argmin(abs(black_pixel_counts - perline * (i + 1))) + 1 
        ideal_radii[N-i-1]= -ideal_radii[N+i+1]
    
    
    radius= np.linspace(-max_radius,max_radius,140)
    bins = np.digitize(radius, ideal_radii,right=True)  #将radius划分到弹性网络的各圈内
    rows, cols=rows-centroid[0], cols-centroid[1]    #减去重心归一化
    accumulator = np.zeros((140, 128))           
    for i in range(len(rows)):
        x=rows[i]
        y=cols[i]
        for k in range(128):
            value=x*coses[k]+y*sins[k]
            j = np.argmin(np.abs(radius - value))      #对每个黑色像素点在每个theta上循环计算半径值，并归到radius轴上离它最近的位置
            accumulator[j,k]+=1                        #这一部分循环像素点(500*500图像可能有20万次上下）对资源的消耗最大
           
    
            
    
    
    accumulator_max=np.zeros(16*7) #通过将(140,128)维的累计器分成(16,7)组，并通过在每组内取最大值的方式降维到(16,7)
    for i in range(16):
        if np.sum(bins == i+1)==0:
                accumulator_max[i*7:(i+1)*7]=0 #如果弹性网络划分出来了一个空集，那么将这个区域的累计器直接置零；避免索引出一个空集，
                continue                              #然后对空集无法取np.max，从而报错
        for j in range(7):
                max_value=np.max(accumulator[bins == i + 1,theta_intervals[j]:theta_intervals[j+1]])
                accumulator_max[i*7+j]=max_value  #注意bins的划分包括(负无穷，最小划分点)和(最大划分点，正无穷)，因此0
                                                   #并不代表最小划分点右侧第一个区间，1才是，所以bin==i+1
    return accumulator_max


# character-sift-特征提取6

In [9]:
def sobel_gradients(image):
    # 计算 Sobel 梯度
    grad_x = sobel(image, axis=0, mode='constant')
    grad_y = sobel(image, axis=1, mode='constant')
    
    # 初始化特征图
    feature_map = np.zeros((image.shape[0], image.shape[1], 8))
    
    # 梯度向量的集合
    grad_vectors = np.stack((grad_x, grad_y), axis=-1)
    
    # 归一化梯度向量
    norms = np.linalg.norm(grad_vectors, axis=2, keepdims=True)
    norms[norms == 0] = 1  # 避免零梯度导致的除零错误
    normalized_grad_vectors = grad_vectors / norms
    
    # 定义指定角度的8个单位向量
    angles = np.deg2rad([0, 45, 90, 135, 180, 225, 270, 315])
    unit_vectors = np.array([[np.cos(angle), np.sin(angle)] for angle in angles])

    # 计算余弦相似度
    cos_similarity = np.tensordot(normalized_grad_vectors, unit_vectors, axes=([2], [1]))
    
    # 为每个梯度向量找到最匹配的两个单位向量
    best_match_indices = np.argsort(-cos_similarity, axis=2)[:, :, :2]
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            indices = best_match_indices[i, j]
            nearest_vectors = unit_vectors[indices]
            grad_vector = grad_vectors[i, j]

            if np.all(grad_vector == 0):
                continue

            # 计算两个最接近向量的系数
            coefficients = np.linalg.solve(nearest_vectors.T, grad_vector)
            feature_map[i, j, indices] = coefficients
    
    return feature_map #500*500*8

def cutline(image, t=6):
    allPoint = np.sum(image)
    if allPoint == 0:
        return np.zeros(t**2)
    perline = allPoint / t
    lie = np.sum(image, axis=0)
    hang = np.sum(image, axis=1)

    for i in range(1, len(lie)):
        lie[i] = lie[i-1] + lie[i]
    for i in range(1, len(hang)):
        hang[i] = hang[i-1] + hang[i]

    lieline = np.array([0] * t + [image.shape[1]])
    hangline = np.array([0] * t + [image.shape[0]])

    for i in range(0, t-1):
        lieline[i+1] = np.argmin(abs(lie - perline * (i + 1))) + 1
        hangline[i+1] = np.argmin(abs(hang - perline * (i + 1))) + 1

    return lieline, hangline


def compute_weighted_feature_map(feature_map, binary_image, t=7): #binary_image 500*500二值图像
    # 使用 cutline 函数获取 7x7 网格的分割线
    lieline, hangline = cutline(binary_image, t=t)

    # 为 7x7 网格初始化特征图
    grid_feature_map = np.zeros((t, t, 8))
    cell_widths = np.diff(lieline)
    cell_heights = np.diff(hangline)
    
    
    for i in range(t):
        for j in range(t):
            
            cell_height=cell_heights[i]
            cell_width=cell_widths[j]
            
            Y=np.arange(hangline[i],hangline[i+1])
            X=np.arange(lieline[j],lieline[j+1])
            
            center_y = (hangline[i] + hangline[i + 1]) // 2
            center_x = (lieline[j] + lieline[j + 1]) // 2
            
            W_X=0.5+(0.5*cell_width-np.abs(X-center_x))/cell_width
            W_X=W_X[np.newaxis,:,np.newaxis]
            W_X_adj=1-W_X
            W_Y=0.5+(0.5*cell_height-np.abs(Y-center_y))/cell_height
            W_Y=W_Y[:,np.newaxis,np.newaxis]
            W_Y_adj=1-W_Y
            
            region1=feature_map[hangline[i]:center_y,lieline[j]:center_x]
            region2=feature_map[hangline[i]:center_y,center_x:lieline[j+1]]
            region3=feature_map[center_y:hangline[i+1],lieline[j]:center_x]
            region4=feature_map[center_y:hangline[i+1],center_x:lieline[j+1]]
            
            
            grid_feature_map[i,j] += np.sum(region1*W_Y[0:center_y-hangline[i]]*W_X\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
            
            grid_feature_map[i,j] += np.sum(region2*W_Y[0:center_y-hangline[i]]*W_X\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
            
            grid_feature_map[i,j] += np.sum(region3*W_Y[center_y-hangline[i]:]*W_X\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
            
            grid_feature_map[i,j] += np.sum(region4*W_Y[center_y-hangline[i]:]*W_X\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
            
            
            
            if j>0:
                grid_feature_map[i,j-1] += np.sum(region1*W_Y[0:center_y-hangline[i]]*W_X_adj\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
                grid_feature_map[i,j-1]+= np.sum(region3*W_Y[center_y-hangline[i]:]*W_X_adj\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
            
            if i>0:
                grid_feature_map[i-1,j] += np.sum(region1*W_Y_adj[0:center_y-hangline[i]]*W_X\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
                
                grid_feature_map[i-1,j] += np.sum(region2*W_Y_adj[0:center_y-hangline[i]]*W_X\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
            
            if j<t-1:
                grid_feature_map[i,j+1] += np.sum(region2*W_Y[0:center_y-hangline[i]]*W_X_adj\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
                
                grid_feature_map[i,j+1]+= np.sum(region4*W_Y[center_y-hangline[i]:]*W_X_adj\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
                
                
            if i<t-1:
                grid_feature_map[i+1,j] += np.sum(region3*W_Y_adj[center_y-hangline[i]:]*W_X\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
                
                grid_feature_map[i+1,j] += np.sum(region4*W_Y_adj[center_y-hangline[i]:]*W_X\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
                
            
            if i>0 and j>0:
                grid_feature_map[i-1,j-1] += np.sum(region1*W_Y_adj[0:center_y-hangline[i]]*W_X_adj\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
                
            if i>0 and j<t-1:
                grid_feature_map[i-1,j+1] += np.sum(region2*W_Y_adj[0:center_y-hangline[i]]*W_X_adj\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
                
            if i<t-1 and j>0:
                grid_feature_map[i+1,j-1] += np.sum(region3*W_Y_adj[center_y-hangline[i]:]*W_X_adj\
                                                  [:,:center_x-lieline[j]],axis=(0,1))
                
            if i<t-1 and j<t-1:
                grid_feature_map[i+1,j+1] += np.sum(region4*W_Y_adj[center_y-hangline[i]:]*W_X_adj\
                                                  [:,center_x-lieline[j]:],axis=(0,1))
                
                
    norms = np.linalg.norm(grid_feature_map, axis=2, keepdims=True)
    # 使用L2范数来进行归一化，避免除以0
    grid_feature_map = grid_feature_map / np.where(norms == 0, 1, norms)
    grid_feature_map = grid_feature_map.flatten()
    if np.isnan(grid_feature_map).any() or np.isinf(grid_feature_map).any():
        print("数组中包含 NaN 或无穷大值！")
    grid_feature_map[grid_feature_map < 0] = 0  # 确保没有负数
    grid_feature_map = np.nan_to_num(grid_feature_map)  # 替换 NaN 或无穷值
    grid_feature_map = grid_feature_map**(0.4)  # 执行幂次运算
    
            
    return grid_feature_map

# 80维内外轮廓特征+20维特征词袋embedding-特征提取7

In [10]:
#见https://github.com/caojiaolong/Fu-Character-Recognition

# 特征提取函数

In [11]:
def makeFeature(method,path,save=True,output_path=None):
    filenames = [f for f in os.listdir(path) if f.endswith('.png')]
    filenames.sort()
    all_features = []
    times = 0
    for filename in tqdm(filenames):
        imagePath = os.path.join(path,filename)
        image = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(filename)
            break
            
        if method == "hough": #hough计算过于昂贵，故下采样
            image = cv2.resize(image, (140,140), interpolation=cv2.INTER_NEAREST)
        
        _,binary_resized_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
        img = (image == 0).astype(int)
        
        if method == "ZBH":
            feature = statistics_strokes_extraction(img)
        elif method == "NWHog":
            feature = extract_feature(img)
        elif method == "Gabor":
            feature = gabor_extraction(img,filters)
        elif method == "STXGabor":
            feature = gabor_ZiBiHua(img)
        elif method == "hough":
            feature=hough_extraction(img)
        elif method == "Character-SIFT":
            sobel_feature_map = sobel_gradients(img)
            feature = compute_weighted_feature_map(sobel_feature_map, img, t=7)
        else:
            print("WRONG METHOD!")
            break

        all_features.append(feature)
        times += 1
    print("共得到%d个图片的特征" % times)
    features_array=np.array(all_features)
    
    if save==True:
        np.save(output_path, features_array)
        
    return features_array

In [12]:
#makeFeature(method="Gabor",path=r"D:\\data\\a",save=True,output_path="D:\\data\\a\\gabor.npy")
#makeFeature(method="ZBH",path=r"D:\\data\\a",save=True,output_path="D:\\data\\a\\ZBH.npy")